# Notebook for generating training data for NeRF from ROSBAG

In [ ]:
# Load libraries for 3D visualization, point cloud manipulation, tensor operations
from vedo import * # powerful visualization toolkit for 3D rendering (built on top of VTK)
from ipyvtklink.viewer import ViewInteractiveWidget # enables VTK 3D renderers to be shown inside Jupyter Notebooks
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import tensorflow as tf

# Limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 10*1024 # Set GPU memory limit to 10 GB
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)

# Load custom modules for spherical alignment and NeRF training
from ICET_spherical import ICET # custom module for Iterative Closest Embedded Transform (used for aligning point clouds)
from scipy.spatial.transform import Rotation as R
import copy

from matplotlib import pyplot as p
from nerf_utils import *
from coarse_network_utils import*

# Enable IPython magic for automatic code reload and autosave
%load_ext autoreload
%autoreload 2
%autosave 180

# Download dataset from external source

ex: https://ori-drs.github.io/newer-college-dataset/

Move to corresponding folder in ../data/


# Export point clouds from ROSBAG to csv file  

1. edit <bag2mapframe.py> to save generated point clouds to correct directory


2. Move <bag2mapframe.py> to ros directory on your machine and run it with:

```
mv bag2mapframe.py ~/catkin_ws/src/bagconverter
cd ~/catkin_ws
catkin_make
cd src/bagconverter
roscore
python3 bag2mapframe.py
```
3. play rosbag 

```
rosbag play -r 0.1 myBag.bag
```

# Load and filter ground truth pose data

In [ ]:
# Load and process GT poses
dir_name = "~/PLINK/data/NewerCollegeDataset/"
experiment_name = "01_short_experiment-20230331T172433Z-009/01_short_experiment/"
fn_gt = dir_name + experiment_name + "ground_truth/registered_poses.csv"

# Load timestamped pose data from CSV
# File format: sec, nsec, x, y, z, qx, qy, qz, qw
gt = np.loadtxt(fn_gt, delimiter=',',skiprows = 1)

# Extract individual components
seconds = gt[:, 0] # Seconds part of timestamp
nano_seconds = gt[:, 1] # Nanoseconds part of timestamp
xyz = gt[:, 2:5] # Position: x, y, z
qxyzw = gt[:, 5:] # Orientation: quaternion (qx, qy, qz, qw)

# Total number of poses in the file
num_poses = qxyzw.shape[0]

# Convert each pose to a 4x4 transformation matrix
# Create a batch of 4x4 identity matrices, one per pose
sensor_poses = np.eye(4, dtype=np.float64).reshape(1, 4, 4).repeat(num_poses, axis=0)
sensor_poses[:, :3, :3] = R.from_quat(qxyzw).as_matrix() # Fill in rotation from quaternion
sensor_poses[:, :3, 3] = xyz # Fill in translation (position)

# Transform poses from camera frame to LiDAR frame
T_CL = np.eye(4, dtype=np.float32) # Transform from Camera to LiDAR (based on calibration)
T_CL[:3, :3] = R.from_quat([0.0, 0.0, 0.924, 0.383]).as_matrix() # Rotation part (manually tuned / externally calibrated)
T_CL[:3, 3] = np.array([-0.084, -0.025, 0.050], dtype=np.float32) # Translation part (manual offset between sensor origins)
sensor_poses = np.einsum("nij,jk->nik", sensor_poses, T_CL) # Apply T_CL to all poses using batch matrix multiplication

# Normalize all poses relative to initial pose
initial_pose = np.linalg.inv(sensor_poses[0]) # Save initial pose (inverse of first frame) to rebase all others
poses_timestamps = seconds * 10e9 + nano_seconds # Compose timestamps (as int-like values) for future synchronization; scale seconds to nanoseconds
sensor_poses = np.einsum("ij,njk->nik", np.linalg.inv(sensor_poses[0]), sensor_poses) # Rebase all poses so that the first pose becomes the origin; TRY COMMENTING OUT...

# Compute body-frame linear velocity (used for motion distortion correction)
vel_world_frame = np.diff(sensor_poses[:,:3,-1], axis = 0) # Calculate velocity in world frame: difference in translation across time
vel_body_frame = np.linalg.pinv(sensor_poses[1:,:3,:3]) @ vel_world_frame[:,:,None] # Convert world-frame velocity to body-frame using pseudo-inverse of rotation matrices
vel_body_frame = vel_body_frame[:,:,0] # Remove singleton dimension

# Smooth velocity using moving average to reduce noise
# Helper function: compute simple moving average over window size `n`
def moving_average(a, n=10):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

# Apply moving average smoothing (window=50) to each velocity component
window=50
MAx = moving_average(vel_body_frame[:,0], n = window)
MAy = moving_average(vel_body_frame[:,1], n = window)
MAz = moving_average(vel_body_frame[:,2], n = window)

# Stack smoothed velocity components back together
vel_body_frame = np.array([MAx, MAy, MAz]).T

# Compute approximate angular velocity from Euler angle changes
rot_vel_euls = np.diff(R.from_matrix(sensor_poses[:,:3,:3]).as_euler('xyz'), axis = 0) # Convert rotation matrices to Euler angles and take time-difference
# Correct wrap-around issues: if angular diff > π, set to 0 (could refine with proper unwrapping)
idx = np.argwhere(rot_vel_euls > (np.pi))  
rot_vel_euls[idx] = 0
idx = np.argwhere(rot_vel_euls < (-np.pi))
rot_vel_euls[idx] = 0

# Load HD map from .ply mesh
# Courtyard section of Newer College Dataset (high-res 1 cm mesh)
pl = '~/PLINK/data/NewerCollegeDataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'

# Alternate section (forest) — comment above line and uncomment below to switch
# pl = '~/PLINK/data/NewerCollegeDataset/new-college-29-01-2020-1cm-resolution-5thSection.ply'

# Load the mesh and extract only the vertex positions (i.e., 3D points)
HD_map = trimesh.load(pl).vertices

# Downsample HD map by keeping every N-th point (reduces memory & speeds up ICP)
show_nth = 5
submap = HD_map[::show_nth]

# Generate training data

In [ ]:
# LiDAR Image Generation Parameters
n_images = 24 # Total number of LiDAR scans to process
n_rots = 128 # Number of horizontal image patches covering 360°
n_vert_patches = 1 # Vertical subdivisions between min and max vertical angles (phimin, phimax)
useICET = True # Whether to use ICET (point cloud registration) for pose correction (turn off for some datasets like foliage)

# Split each LiDAR image into (n_rots x n_vert_patches) patches via num pixels // num patches 
image_width = 1024//n_rots # Width of each image patch (pixels per horizontal division)
image_height = 64//n_vert_patches # Height of each image patch (pixels per vertical division)
shrink_factor = 0.005 # Scales the world to fit inside a compact coordinate system (used in courtyard mapping)

n_cols_to_skip = n_rots // 8 # Mask out patches at edges of scans (e.g., where operator appears in frame)

# Sensor-specific field of view calibration (Ouster OS1-64)
phimin = np.deg2rad(-15.594) # Minimum vertical angle (in radians)
phimax = np.deg2rad(17.743) # Maximum vertical angle (in radians)
vert_fov = np.rad2deg(phimax-phimin) # Total vertical field of view in degrees

# Allocate arrays for training outputs
poses = np.zeros([n_images*n_rots*n_vert_patches,4,4]) # 4x4 camera pose for each patch (rotation + translation)

# Each image contains two channels:
#   - depth (channel 0)
#   - raydrop mask (channel 1, 1 if valid, 0 if missing)
images = np.ones([n_images*n_rots*n_vert_patches, 64//n_vert_patches, 1024//n_rots, 2])

# [n total "patches", patch height, patch width, xyz]; ray origin and direction for each pixel (for NeRF ray marching)
rays_o_all = np.zeros([n_images*n_rots*n_vert_patches, 64//n_vert_patches, 1024//n_rots, 3]) # origin of each ray in 3D space for each pixel in each patch
rays_d_all = np.zeros([n_images*n_rots*n_vert_patches, 64//n_vert_patches, 1024//n_rots, 3]) # direction of each ray in 3D space for each pixel in each patch

H, W = images.shape[1:3]

redfix_hist = np.zeros([n_images,4,4]) # Save ICET correction transforms for each frame (applied to raw LiDAR)

# Loop over LiDAR frames and extract ray/depth data
for i in range(n_images):
    print(i) 
    idx = i*50 + 7650  # Index into point cloud sequence (adjust depending on dataset split)
    fn1 = "~/PLINK/data/NewerCollegeDataset/01_Short_Experiment/point_clouds/frame_" + str(idx) + ".npy"
    pc1 = np.load(fn1)

    # Apply motion distortion correction using estimated velocity
    # m_hat = estimated motion between frames: [dx, dy, dz, droll, dpitch, dyaw]
    m_hat = np.array([-vel_body_frame[idx,0],
                      -vel_body_frame[idx,1],
                      -vel_body_frame[idx,2],
#                       -rot_vel_euls[idx,0], (Optionally include angular correction from rot_vel_euls)
#                       -rot_vel_euls[idx,1],
#                       -rot_vel_euls[idx,2]
                      0.,0.,0.
                     ])   
    pc1 = apply_motion_profile(pc1, m_hat, period_lidar=1.)  # Correct LiDAR sweep using motion profile
    pc1 = np.flip(pc1, axis = 0) # Flip row ordering (sensor-specific correction)

    # Register undistorted PC against HD Map using ICET to correct issues in ground truth
    if useICET:
        # Transform HD map points to LiDAR frame
        submap_in_pc1_frame = (np.linalg.pinv(sensor_poses[idx]) @ initial_pose @ np.append(submap, np.ones([len(submap),1]), axis =1).T).T #test
        submap_in_pc1_frame = submap_in_pc1_frame[:,:3] # Drop homogeneous column

        # Use ICET to refine alignment of submap and scan
        initial_guess = tf.constant([0.,0.,0.,0.,0.,0.])
         # Run ICET algorithm for fine registration
        it = ICET(cloud1 = submap_in_pc1_frame, cloud2 = pc1, fid = 50, niter = 8, 
           draw = False, group = 2, RM = False, DNN_filter = False, x0 = initial_guess)

        # Transform original and corrected scans to map frame for inspection
        pc1_in_map_frame = (initial_pose @ sensor_poses[idx] @ np.append(pc1, np.ones([len(pc1),1]), axis =1).T).T #test
        pc1_in_map_frame = pc1_in_map_frame[:,:3]

        pc1_corrected_in_map_frame = (initial_pose @ sensor_poses[idx] @ np.append(it.cloud2_tensor.numpy(), np.ones([len(it.cloud2_tensor.numpy()),1]), axis =1).T).T #test
        pc1_corrected_in_map_frame = pc1_corrected_in_map_frame[:,:3]    

        # Build correction transformation matrix from ICET output
        redFix = np.eye(4)
        redFix[:3,-1] = it.X[:3] # Apply translation from ICET
        redFix[:3,:3] = redFix[:3,:3] @ R.from_euler('xyz', [it.X[3], it.X[4], it.X[5]]).as_matrix() # Apply rotation
        redfix_hist[i] = redFix # Save correction matrix

        # Apply ICET correction and transform back into LiDAR frame
        redScanFixed = (redFix @ np.append(pc1, np.ones([len(pc1),1]), axis =1).T).T
        redScanFixed = (sensor_poses[idx] @ np.append(redScanFixed[:,:3], np.ones([len(redScanFixed),1]), axis =1).T).T
 
    else:
        # No correction — just store identity matrix and original point cloud
        redFix = np.eye(4)
        redfix_hist[i] = redFix
        redScanFixed = (redFix @ np.append(pc1, np.ones([len(pc1),1]), axis =1).T).T
        redScanFixed = (sensor_poses[idx] @ np.append(redScanFixed[:,:3], np.ones([len(redScanFixed),1]), axis =1).T).T

    # Convert corrected point cloud into spherical image
    pc1_spherical = cartesian_to_spherical(pc1).numpy() # (r, theta, phi)
    pcs = np.reshape(pc1_spherical, [-1,64,3]) # Reshape to match 64-row sensor: [n, 64, 3]
    pcs = np.flip(pcs, axis = 1) # Flip vertically (sensor-specific)
    raw_data = pcs[:,:,:] 
    raw_data = np.transpose(pcs, [1,0,2]) # Reorder axes to [rows, cols, features]

    # Destagger depth images (OS1 unit has delay in sensor return bus)
    data = np.zeros([64, 1024])
    for k in range(np.shape(data)[0]//4):  # OS1 sends data in quartets
        data[4*k,1:-8] = raw_data[4*k,9:,0]
        data[4*k+1,1:-2] = raw_data[4*k+1,3:,0]
        data[4*k+2,4:] = raw_data[4*k+2,:-4,0]
        data[4*k+3,10:] = raw_data[4*k+3,:-10,0]
    data = np.flip(data, axis =1) # Reverse horizontal order

    # Generate rays (origin & direction) for this frame
    rotm = sensor_poses[idx] @ redfix_hist[i]
    rotm[0,-1] += 30
    rotm[1,-1] += 30
    rotm[2,-1] += 15 
    rotm[:3,-1] *= shrink_factor #0.02 #0.005 #COURTYARD
    #courtyard
    rotm[0,-1] += 0.01 
    rotm[1,-1] += 0.25 
    rotm[2,-1] += 0.25 #translate above xy plane
#     #forest
#     rotm[0,-1] += 1.2 
#     rotm[1,-1] += 1.25 
#     rotm[2,-1] += 0.25 #translate above xy plane
    ro, rd = get_rays_from_point_cloud(pc1, m_hat, rotm) 

    # Split full scan into ray-image patches
    for j in range(n_rots):
        for k in range(n_vert_patches):    
            # Store ray directions
            rd_in_patch = rd[k*image_height:(k+1)*image_height,j*image_width:(j+1)*image_width, :]
            rays_d_all[k+(j+(i*n_rots))*n_vert_patches,:,:,:] = rd_in_patch

            # Store ray origins
            ro_in_patch = ro[k*image_height:(k+1)*image_height,j*image_width:(j+1)*image_width, :]
            rays_o_all[k+(j+(i*n_rots))*n_vert_patches,:,:,:] = ro_in_patch            

            # Store depth image for patch
            # Crop vertically and horizontally
            pcs = data[k*image_height:(k+1)*image_height,j*image_width:(j+1)*image_width] 
            #save depth information to first channel
            images[k+(j+(i*n_rots))*n_vert_patches,:,:,0] = pcs
            # Create raydrop mask (0 where invalid)
            a = np.argwhere(abs(pcs) < 1)
            images[k+(j+(i*n_rots))*n_vert_patches, a[:,0],a[:,1],1] = 0

            # Get transformation matrix
            # Centers origin at actual origin of HD map 
            # Update pose matrix for this patch
            rotm = sensor_poses[idx] @ redfix_hist[i]

            crop_angle = j*(2*np.pi/n_rots) - np.pi/2 + (np.pi/n_rots)
            #account for the fact that sensor points back and to the left
            rotm_crop = R.from_euler('xyz', [0,0,-crop_angle]).as_matrix() #test
            rotm[:3,:3] = rotm[:3,:3] @ rotm_crop
            rotm[0,-1] += 30
            rotm[1,-1] += 30
            rotm[2,-1] += 15 
            rotm[:3,-1] *= shrink_factor
            images[k+(j+(i*n_rots))*n_vert_patches,:,:,0] *= shrink_factor
            #courtyard
            rotm[0,-1] += 0.01 #shift up just a little
            rotm[1,-1] += 0.25 #shift towards positive x
            rotm[2,-1] += 0.25 #translate above xy plane
#             #forest
#             rotm[0,-1] += 1.2 
#             rotm[1,-1] += 1.25 
#             rotm[2,-1] += 0.25 #translate above xy plane

            poses[k+(j+(i*n_rots))*n_vert_patches] = rotm 

# Remove patches where sensor is occluded by person holding lidar 
# Calculate how many columns of patches we need to skip at the beginning and end of each scan to avoid
bad_idx = np.zeros([0,n_rots - 2*n_cols_to_skip]) # Create an empty array to store indices of patches that need to be removed
a = np.linspace(0,n_rots*n_images*n_vert_patches-1,n_rots*n_images*n_vert_patches) # Create a linear array of all patch indices (total = n_images * n_rots * n_vert_patches)

# Loop through the number of columns to skip on both ends
for i in range(n_vert_patches*n_cols_to_skip):
    bad_i_left = a[i::n_rots*n_vert_patches] # Identify patches in the i-th leftmost column across all scans
    bad_idx = np.append(bad_idx, bad_i_left) # Append these indices to the list of bad indices
    bad_i_right = a[(i+n_vert_patches*(n_rots-n_cols_to_skip))::n_rots*n_vert_patches] # Identify patches in the i-th rightmost column across all scans
    bad_idx = np.append(bad_idx, bad_i_right) # Append these indices as well

bad_idx = np.sort(bad_idx) # Sort the full list of bad patch indices in ascending order
all_idx = np.linspace(0,n_rots*n_images*n_vert_patches-1,n_rots*n_images*n_vert_patches) # Create a full list of all patch indices
good_idx = np.setdiff1d(all_idx, bad_idx).astype(int) # Subtract bad indices from full set to get valid (non-occluded) patch indices

# Keep only valid (non-occluded) patches in each data array
images = images[good_idx,:,:,:] 
poses = poses[good_idx,:,:]
rays_d_all = rays_d_all[good_idx,:,:,:]
rays_o_all = rays_o_all[good_idx,:,:,:]

# Convert data types to float32 for compatibility with training frameworks
images = images.astype(np.float32)
poses = poses.astype(np.float32)

# Visualize training data

draw training data in the same frame using depth images, ray origins (rays_o), 
and view directions (rays_d)

In [ ]:
# Visualization: draw rays and depth points from image patches
def draw_frame_from_rays(disp, n_rots=128, n_vert_patches=1, frameIdx=0, 
                         color = 'red', stitched_map = np.zeros([0,3]) ):
    
    # Compute vertical resolution and elevation angle bins
    phimin = np.deg2rad(-15.594) #took forever to figure this out...
    phimax = np.deg2rad(17.743)
    H = 64 // n_vert_patches
    W = 1024 // n_rots
    vertical_bins = np.linspace(phimin, phimax, n_vert_patches+1)  
    phivals = np.linspace(phimin, phimax, 64) # Used for elevation slicing
    n_cols_to_skip = n_rots // 8 # Mask first/last N columns

    pts1 = np.zeros([1,3]) # Accumulator for all 3D points
    # Loop over each patch for the selected frameIdx (excluding occluded columns)
    for p in range(frameIdx*(n_rots - 2*n_cols_to_skip), (frameIdx + 1 )*(n_rots - 2*n_cols_to_skip)):
        for i in range(n_vert_patches):
            img_i = i
            # Compute elevation range for the current vertical patch
            idx_first=len(phivals) - (img_i%(n_vert_patches))*(64//n_vert_patches)-1 
            idx_second= (len(phivals)- ((img_i+1)%(n_vert_patches))*(64//n_vert_patches))%len(phivals)
            phimin_patch = phivals[idx_first]
            phimax_patch = phivals[idx_second]

            # Retrieve pose and rays for current patch
            pose = poses[i + p*n_vert_patches]
            rays_o = rays_o_all[i + p*n_vert_patches]
            rays_d = rays_d_all[i + p*n_vert_patches]

            # Reconstruct 3D points from rays and depth (projected in space)
            inMap1 = add_patch(rays_o, rays_d, images[i+p*n_vert_patches,:,:,0])

            # Accumulate all 3D points for display
            pts1 = np.append(pts1, inMap1, axis = 0)
        disp.append(Points(rays_o[0,:1,:], r = 15, c = 'purple')) # Debug: show ray origin as a purple sphere

    # Render the projected 3D points (in assigned color, semi-transparent)
    vizPts1 = Points(pts1, c = color, r = 3., alpha = 0.125)
    disp.append(vizPts1)
    
    # Accumulate the 3D points into the stitched map (used across frames)
    stitched_map = np.append(stitched_map, pts1, axis = 0)
    return stitched_map
    
# Set up 3D plotter and visualize training set
plt = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[] # List of items to render

# Allocate memory for accumulated 3D point cloud
stitched_map = np.zeros([0,3])
# Create a range of color intensities for different frames; colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'red']
colors = np.linspace(0.1,0.3,n_images)[:,None] * np.array([[1,1,1]])
# Draw all training frames one by one
for i in range(len(colors)):
    print(i)
    stitched_map = draw_frame_from_rays(disp, n_rots = 128, n_vert_patches=1, 
                                        frameIdx = i, color=colors[i], stitched_map=stitched_map)

# Display the visualization window
plt.show(disp, "Drawing training data from depth images, rays_o, and rays_d")
# Enable interactive 3D exploration in Jupyter Notebook
ViewInteractiveWidget(plt.window)  

# Save training data 

In [ ]:
np.save("~/PLINK/data/NewerCollegeDataset/images.npy", images)
np.save("~/PLINK/data/NewerCollegeDataset/poses.npy", poses)
np.save("~/PLINK/data/NewerCollegeDataset/rays_o.npy", rays_o_all)
np.save("~/PLINK/data/NewerCollegeDataset/rays_d.npy", rays_d_all)